# Spark DataFrames / SQL

En el notebook anterior hemos introducido Spark y el uso de RDD para interactuar con los datos. Tal como comentamos, los RDD permiten trabajar a bajo nivel, siendo más cómodo y eficiente hacer uso de DataFrames y el lenguaje SQL.

## DATAFRAMES

Un DataFrame es una estructura equivalente a una tabla de base de datos relacional, con un motor bien optimizado para el trabajo en un clúster. Los datos se almacenan en filas y columnas y ofrece un conjunto de operaciones para manipular los datos.

El trabajo con DataFrames es más sencillo y eficiente que el procesamiento con RDD, por eso su uso es predominante en los nuevos desarrollos con Spark.

A continuación veremos cómo podemos obtener y persistir DataFrames desde diferentes fuentes y formatos de datos

### Creando Dataframes

El caso más básico es crear un DataFrame a partir de un RDD mediante `toDF`:

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate() # SparkSession de forma programativa
# Creamos un RDD
datos = [("Aitor", 182), ("Pedro", 178), ("Marina", 161)]
rdd = spark.sparkContext.parallelize(datos)
# Creamos un DataFrame y mostramos su esquema
dfRDD = rdd.toDF()
dfRDD.printSchema()
# mediante 'printSchema'  obtenemos un resumen del esquema del DataFrame , donde para cada columna se indica el nombre, el tipo y si admite valores nulos.

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)



Podemos ver como los nombres de las columnas son _1 y _2. Para asignarle un nombre adecuado podemos pasarle una lista con los nombres a la hora de crear el DataFrame:

In [0]:
columnas = ["nombre","altura"]
dfRDD = rdd.toDF(columnas)
dfRDD.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- altura: long (nullable = true)



Si queremos mostrar sus datos, haremos uso del método `show`:

In [0]:
dfRDD.show()
# Obtenemos una vista de los datos en forma de tabla:

+------+------+
|nombre|altura|
+------+------+
| Aitor|   182|
| Pedro|   178|
|Marina|   161|
+------+------+



También podemos crear un DataFrame directamente desde una `SparkSession` sin crear un RDD previamente mediante el método `createDataFrame`:

In [0]:
dfDesdeDatos = spark.createDataFrame(datos, columnas)
dfDesdeDatos.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- altura: long (nullable = true)



### Mostrando los datos

Para los siguientes apartados, supongamos que queremos almacenar ciertos datos de clientes, como son su nombre y apellidos, ciudad y sueldo:

In [0]:
clientes = [
    ("Aitor", "Medrano", "Elche", 3000),
    ("Pedro", "Casas", "Elche", 4000),
    ("Laura", "García", "Elche", 5000), 
    ("Miguel", "Ruiz", "Torrellano", 6000),
    ("Isabel", "Guillén", "Alicante", 7000)
]
columnas = ["nombre","apellidos", "ciudad", "sueldo"]
df = spark.createDataFrame(clientes, columnas)

Para mostrar los datos podemos utilizar el método `show`, al cual le podemos indicar o no la cantidad de registros a recuperar, así como si queremos que los datos se trunquen o no, o si los queremos mostrar en vertical:

In [0]:
df.show(2)

+------+---------+------+------+
|nombre|apellidos|ciudad|sueldo|
+------+---------+------+------+
| Aitor|  Medrano| Elche|  3000|
| Pedro|    Casas| Elche|  4000|
+------+---------+------+------+
only showing top 2 rows



In [0]:
df.show(truncate=False)

+------+---------+----------+------+
|nombre|apellidos|ciudad    |sueldo|
+------+---------+----------+------+
|Aitor |Medrano  |Elche     |3000  |
|Pedro |Casas    |Elche     |4000  |
|Laura |García   |Elche     |5000  |
|Miguel|Ruiz     |Torrellano|6000  |
|Isabel|Guillén  |Alicante  |7000  |
+------+---------+----------+------+



In [0]:
df.show(3, vertical=True)

-RECORD 0------------
 nombre    | Aitor   
 apellidos | Medrano 
 ciudad    | Elche   
 sueldo    | 3000    
-RECORD 1------------
 nombre    | Pedro   
 apellidos | Casas   
 ciudad    | Elche   
 sueldo    | 4000    
-RECORD 2------------
 nombre    | Laura   
 apellidos | García  
 ciudad    | Elche   
 sueldo    | 5000    
only showing top 3 rows



Si sólo queremos recuperar unos pocos datos, podemos hacer uso de `head` o `first` los cuales devuelven objetos `Row`:

In [0]:
df.first()

Out[10]: Row(nombre='Aitor', apellidos='Medrano', ciudad='Elche', sueldo=3000)

In [0]:
df.head()

Out[11]: Row(nombre='Aitor', apellidos='Medrano', ciudad='Elche', sueldo=3000)

In [0]:
df.head(3)

Out[12]: [Row(nombre='Aitor', apellidos='Medrano', ciudad='Elche', sueldo=3000),
 Row(nombre='Pedro', apellidos='Casas', ciudad='Elche', sueldo=4000),
 Row(nombre='Laura', apellidos='García', ciudad='Elche', sueldo=5000)]

Si queremos obtener un valor en concreto, una vez recuperada una fila, podemos acceder a sus columnas:

In [0]:
nom1 = df.first()[0]           # 'Aitor'
nom1

Out[13]: 'Aitor'

In [0]:
nom2 = df.first()["nombre"]    # 'Aitor'
nom2

Out[14]: 'Aitor'

También podemos obtener un sumario de los datos (igual que con Pandas) mediante `describe`:

In [0]:
df.describe()

Out[15]: DataFrame[summary: string, nombre: string, apellidos: string, ciudad: string, sueldo: string]

In [0]:
df.describe().show()

+-------+------+---------+----------+------------------+
|summary|nombre|apellidos|    ciudad|            sueldo|
+-------+------+---------+----------+------------------+
|  count|     5|        5|         5|                 5|
|   mean|  null|     null|      null|            5000.0|
| stddev|  null|     null|      null|1581.1388300841897|
|    min| Aitor|    Casas|  Alicante|              3000|
|    max| Pedro|     Ruiz|Torrellano|              7000|
+-------+------+---------+----------+------------------+



Si únicamente nos interesa saber cuantas filas tiene nuestro DataFrame, podemos hacer uso de `count`:

In [0]:
df.count()  # 5

Out[17]: 5

Por último, como un DataFrame por debajo es un RDD, podemos usar `collect` y `take` conforme necesitemos y recuperar objetos de tipo `Row`:

In [0]:
df.collect()

Out[18]: [Row(nombre='Aitor', apellidos='Medrano', ciudad='Elche', sueldo=3000),
 Row(nombre='Pedro', apellidos='Casas', ciudad='Elche', sueldo=4000),
 Row(nombre='Laura', apellidos='García', ciudad='Elche', sueldo=5000),
 Row(nombre='Miguel', apellidos='Ruiz', ciudad='Torrellano', sueldo=6000),
 Row(nombre='Isabel', apellidos='Guillén', ciudad='Alicante', sueldo=7000)]

In [0]:
df.take(2)

Out[19]: [Row(nombre='Aitor', apellidos='Medrano', ciudad='Elche', sueldo=3000),
 Row(nombre='Pedro', apellidos='Casas', ciudad='Elche', sueldo=4000)]

In [0]:
nom = df.collect()[0][0]        # 'Aitor'
nom

Out[20]: 'Aitor'

### Cargando diferentes formatos

Lo más usual es cargar los datos desde una archivo externo. Para ello, mediante el API de `DataFrameReader` cargaremos los datos directamente en un Dataframe mediante diferentes métodos dependiendo del formato (admite tanto el nombre de un recurso como una ruta de una carpeta).

Para cada formato, existe un método corto que se llama como el formato en sí, y un método general donde mediante format indicamos el formato y que finaliza con el método `load` siempre dentro de `spark.read`:

**CSV**
```<python>
dfCSV = spark.read.csv("datos.csv")
dfCSV = spark.read.csv("datos/*.csv")   # Una carpeta entera
dfCSV = spark.read.option("sep", ";").csv("datos.csv")
dfCSV = spark.read.option("header", "true").csv("datos.csv")
dfCSV = spark.read.option("header", True).option("inferSchema", True).csv("datos.csv")
dfCSV = spark.read.options(sep=";", header=True, inferSchema=True).csv("pdi_sales.csv")
dfCSV = spark.read.format("csv").load("datos.csv") 
dfCSV = spark.read.load(path="datos.csv", format="csv", header="true", sep=";", inferSchema="true")

``` 



**txt**
```<Python>
dfTXT = spark.read.text("datos.txt")
# cada fichero se lee entero como un registro
dfTXT = spark.read.option("wholetext", true).text("datos/")

dfTXT = spark.read.format("txt").load("datos.txt")
```

**JSON**
```<Python>
dfJSON = spark.read.json("datos.json")
dfJSON = spark.read.format("json").load("datos.json")
```

**parquet**
```<Python>
dfParquet = spark.read.parquet("datos.parquet")
dfParquet = spark.read.format("parquet").load("datos.parquet")
```

Mas información en la [documentación](https://spark.apache.org/docs/latest/sql-data-sources-parquet.html) oficial

**AVRO**
La fuente de datos en formato Avro se incluye como un módulo externo, y por lo tanto, para poder leer o escribir datos en dicho formato, previamente hemos de cargar una librería.
Para ello, al arrancar PySpark, le pasaremos como parámetro `--packages org.apache.spark:spark-avro_2.12:3.3.1`:  

`pyspark --packages org.apache.spark:spark-avro_2.12:3.3.1`  

Una vez arrancado, ya podemos leer y escribir datos en formato Avro de forma similar al resto:  

```
df = spark.read.format("avro").load("datos.avro")
df.write.format("avro").save("archivo.avro")
```  

La librería también nos permite convertir columnas y estructuras de datos con las operaciones `to_avro()` y `from_avro()`. Más información en la [documentación oficial](https://spark.apache.org/docs/latest/sql-data-sources-avro.html).

**Actividad:** Con un fichero cualquiera en `.avro` probar el código anterior e imprimir el contenido de dicho fichero

En nuestro conjunto de datos, contamos con un archivo llamado `weather_data.avro`, el cual utilizaremos para esta práctica. Al cargar y visualizar su contenido, no surgen inconvenientes. Sin embargo, al momento de escribir los datos de nuevo, pueden aparecer problemas debido a los caracteres especiales en los nombres de las columnas. Para evitar estos conflictos, primero realizaremos una limpieza de dichos caracteres antes de guardar el archivo nuevamente.

In [0]:
from pyspark.sql.functions import col

# Leemos.
df = spark.read.format("avro").load("/FileStore/dataframes-spark/weather_data.avro")
df.show()

# Renombramos las columnas reemplazando espacios y barras "/" por guiones bajos "_"
df_cleaned = df.select([col(c).alias(c.replace(" ", "_").replace("/", "_")) for c in df.columns])

# Verificamos el cambio
df_cleaned.show()

# Escribimos
df_cleaned.write.format("avro").save("/FileStore/dataframes-spark/new_weather_data.avro")

+--------------+------+---------------+------+------------+-------------+---------+--------------------+
|     Date/Time|Temp_C|DewPoint Temp_C|RelHum|WindSpeed_km|Visibility_km|Press_kPa|             Weather|
+--------------+------+---------------+------+------------+-------------+---------+--------------------+
| 1/1/2012 0:00|  -2.0|           -3.9|    86|           4|          8.0|   101.24|                 Fog|
| 1/1/2012 1:00|  -1.8|           -3.7|    87|           4|          8.0|   101.24|                 Fog|
| 1/1/2012 3:00|  -1.5|           -3.2|    88|           6|          4.0|   101.27|Freezing Drizzle,Fog|
| 1/1/2012 4:00|  -1.5|           -3.3|    88|           7|          4.8|   101.23|                 Fog|
| 1/1/2012 5:00|  -1.4|           -3.3|    87|           9|          6.4|   101.27|                 Fog|
| 1/1/2012 6:00|  -1.5|           -3.1|    89|           7|          6.4|   101.29|                 Fog|
| 1/1/2012 7:00|  -1.4|           -3.6|    85|         

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3629425818905065>:14
     11 df_cleaned.show()
     13 # Escribimos
---> 14 df_cleaned.write.format("avro").save("/FileStore/dataframes-spark/new_weather_data.avro")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1397, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1395     self._jwrite.save()
   1396 else:
-> 1397     self._jwrite.save(path)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.p

### Persistiendo diferentes formatos  

Si lo que queremos es persistir los datos, en vez de `read`, utilizaremos `write` (de manera que obtenemos un `DataFrameWriter`) y si usamos la forma general usaremos el método `save`:

**CSV**
```<python>
dfCSV.write.csv("datos.csv")
dfCSV.write.format("csv").save("datos.csv")
dfCSV.write.format("csv").mode("overwrite").save("datos.csv")
```

**TXT**
```<python>
dfTXT.write.text("datos.txt")
dfTXT.write.option("lineSep",";").text("datos.txt")
dfTXT.write.format("txt").save("datos.txt")
```

**JSON**
```<python>
dfJSON.write.json("datos.json")
dfJSON.write.format("json").save("datos.json")
```

**Parquet**
```<python>
dfParquet.write.parquet("datos.parquet")
dfParquet.write.mode("overwrite").partitionBy("fecha").parquet("datos/")
dfParquet.write.format("parquet").save("datos.parquet")
```

Más información en la[ documentación oficial](https://spark.apache.org/docs/latest/sql-data-sources-parquet.html)

> Por cada partición, Spark generará un archivo de salida. Recuerda que podemos reducir el `número de particiones` mediante `coalesce` o `repartition`.

Una vez vista la sintaxis, vamos a ver un ejemplo completo de lectura de un archivo CSV (el archivo `pdi_sales.csv`) que está almacenado en HDFS y que tras leerlo, lo guardamos como JSON de nuevo en HDFS:

> **Usar DBFS en vez de HDFS**. El siguiente código esta hecho para HDFS. Modificarlo para que pueda ser ejecutado en DBFS. 

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("s8a-dataframe-csv").getOrCreate()

# Lectura de CSV con el ; como separador de columnas y con encabezado
df = spark.read.option("delimiter",";").option("header", "true").csv("hdfs://iabd-virtualbox:9000/user/iabd/pdi_sales.csv")

# df.printSchema()

df.write.json("hdfs://iabd-virtualbox:9000/user/iabd/pdi_sales_json")

### Comprimiendo los datos

Para configurar el algoritmo de compresión, si los datos están en Parquet o Avro, a nivel de la sesión de Spark, podemos realizar su configuración:

```<python>
spark.setConf("spark.sql.parquet.compression.codec","snappy")
spark.setConf("spark.sql.parquet.compression.codec","none")
spark.setConf("spark.sql.avro.compression.codec","snappy")
```

Si sólo queremos hacerlo para una operación en particular, para cada lectura/escritura le añadimos `.option("compression", "algoritmo")`. Por ejemplo:

In [0]:
# Cargar el archivo CSV de ventas (pdi_sales.csv) con compresión Snappy
dfVentas = spark.read.option("compression", "snappy").option("delimiter", ";").option("header", "true").csv("/FileStore/dataframes-spark/pdi_sales.csv")

# Guardar el DataFrame de ventas en formato Parquet con compresión Snappy
dfVentas.write.option("compression", "snappy").parquet("/FileStore/dataframes-spark/ventas.parquet")

# Leer el archivo Parquet de ventas
dfVentasParquet = spark.read.option("compression", "snappy").parquet("/FileStore/dataframes-spark/ventas.parquet")

# Guardar el DataFrame de ventas en formato Avro con compresión Snappy
dfVentas.write.option("compression", "snappy").format("avro").save("/FileStore/dataframes-spark/ventas.avro")

In [0]:
dfVentas.show()

In [0]:
# Ver el tamaño del archivo en DBFS
dbutils.fs.ls("/FileStore/dataframes-spark/")

## DATOS Y ESQUEMAS 

El esquema completo de un DataFrame se modela mediante un `StructType`, el cual contiene una colección de objetos `StructField`. Así pues, cada columna de un DataFrame de Spark se modela mediante un objeto `StructField` indicando su nombre, tipo y gestión de los nulos.

Hemos visto que al crear un DataFrame desde un archivo externo, podemos inferir el esquema. Si queremos crear un DataFrame desde un esquema propio utilizaremos los tipos `StructType`, `StructField`, así como `StringType`, `IntegerType` o el tipo necesario para cada columna. Para ello, primero hemos de importarlos (como puedes observar, estas clases pertenecen a las librerías SQL de PySpark):

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

> **Tipos**.  
> Además de cadenas y enteros, flotantes (`FloatType`) o dobles (`DoubleType`), tenemos tipos booleanos (`BooleanType`), de fecha (`DateType` y `TimestampType`), así como tipos complejos como `ArrayType`, `MapType` y `StructType`. Para más información, consultar la [documentación oficial](https://spark.apache.org/docs/latest/sql-ref-datatypes.html).

Volvamos al ejemplo anterior donde tenemos ciertos datos de clientes, como son su nombre y apellidos, ciudad y sueldo:

In [0]:
clientes = [
    ("Aitor", "Medrano", "Elche", 3000),
    ("Pedro", "Casas", "Elche", 4000),
    ("Laura", "García", "Elche", 5000), 
    ("Miguel", "Ruiz", "Torrellano", 6000),
    ("Isabel", "Guillén", "Alicante", 7000)
]

Para esta estructura, definiremos un esquema con los campos, indicando para cada uno de ellos su nombre, tipo y si admite valores nulos:

In [0]:
esquema = StructType([
    StructField("nombre", StringType(), False),
    StructField("apellidos", StringType(), False),
    StructField("ciudad", StringType(), True),
    StructField("sueldo", IntegerType(), False)
])

A continuación ya podemos crear un DataFrame con datos propios que cumplen un esquema haciendo uso del método `createDataFrame`:

In [0]:
df = spark.createDataFrame(data=clientes, schema=esquema)
df.printSchema()

In [0]:
df.show(truncate=False)

Si lo que queremos es asignarle un esquema a un DataFrame que vamos a leer desde una fuente de datos externa, hemos de emplear el método `schema`:

```<Python>
dfClientes = spark.read.option("header", True).schema(esquema).csv("clientes.csv")
```

> **Rendimiento y esquema**  
> La inferencia de los tipos de los datos es un proceso computacionalmente costoso. Por ello, si nuestro conjunto de datos es grande, es muy recomendable crear el esquema de forma programativa y configurarlo en la carga de datos. Se recomienda la lectura del artículo [Using schemas to speed up reading into Spark DataFrames](https://t-redactyl.io/blog/2020/08/using-schemas-to-speed-up-reading-into-spark-dataframes.html).

> **Actividad** En otro notebook hacer el ejemplo que se muestra en el artículo que se sugiere arriba. La dataset se encuentra en [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/339/taxi+service+trajectory+prediction+challenge+ecml+pkdd+2015) con el nombre `train.csv.zip`

Respecto al esquema, tenemos diferentes propiedades como columns, dtypes y schema con las que obtener su información:

In [0]:
df.columns

In [0]:
df.dtypes

In [0]:
df.schema

Si una vez hemos cargado un DataFrame queremos cambiar el tipo de una de sus columnas, podemos hacer uso del método `withColumn`:

In [0]:
# Forma larga
from pyspark.sql.types import DoubleType
df = df.withColumn("sueldo", df.sueldo.cast(DoubleType()))

In [0]:
# Forma corta
df = df.withColumn("sueldo", df.sueldo.cast("double"))


In [0]:
# df = df.withColumn("fnac", to_date(df.Date, "M/d/yyy"))

> **Errores al leer datos**  
> Si tenemos un error al leer un dato que contiene un tipo no esperado, por defecto, Spark lanzará una excepción y se detendrá la lectura.  
> Si queremos que asigne los tipos a los campos pero que no los valide, podemos pasarle el parámetro extra verifySchema a False al crear un DataFrame mediante spark.createDataFrame o enforceSchema también a False al cargar desde una fuente externa mediante spark.read, de manera que los datos que no concuerden con el tipo se quedarán nulos, vacíos o con valor 0, dependiendo del tipo de dato que tiene asignada la columna en el esquema.  
`dfClientes = spark.read.option("header", True).option("enforceSchema",False).schema(esquema).csv("clientes.csv")`

## DATAFRAME API

Una vez tenemos un DataFrame podemos trabajar con los datos mediante un conjunto de operaciones estructuradas, muy similares al lenguaje relacional. Estas operaciones también se clasifican en transformaciones y acciones, recordando que las transformaciones utilizan una evaluación perezosa.  
Es muy importante tener en cuenta que todas las operaciones que vamos a realizar a continuación son immutables, es decir, nunca van a modificar el DataFrame sobre el que realizamos la transformación. Así pues, realizaremos encadenamiento de transformaciones (transformation chaining) o asignaremos el resultado a un nuevo DataFrame.

> **Preparación**
> Para los siguientes apartados, vamos a trabajar sobre el siguiente DataFrame con el fichero de [ventas](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EW5mbeDvxFpKkePpayR4A6gBmrh_dpG54CxcqODEnQ7hMw?e=RixWQy) que se ha utilizado en casos anteriores:

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("s8a-dataframes-api").getOrCreate()
# Lectura de CSV con el ; como separador de columnas y con encabezado
df = spark.read.option("sep",";").option("header", "true").option("inferSchema", "true").csv("/FileStorepdi_sales_small.csv")
df.printSchema()

### Proyectando

La operación [select](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.select.html) permite indicar las columnas a recuperar pasándolas como parámetros:

In [0]:
# Consulta de columnas
df.select("ProductID","Revenue").show(3)

También podemos realizar cálculos (referenciando a los campos con `nombreDataframe.nombreColumna`) sobre las columnas y crear un alias (operación asociada a un campo):

In [0]:
# Calculo y creación del alias
df.select(df.ProductID,(df.Revenue+10).alias("VentasMas10")).show(3)

Si tenemos un DataFrame con un gran número de columnas y queremos recuperarlas todas a excepción de unas pocas, es más cómodo utilizar la transformación [drop](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.drop.html), la cual funciona de manera opuesta a `select`, es decir, indicando las columnas que queremos quitar del resultado:

In [0]:
# Obtenemos el mismo resultado
df.select("ProductID", "Date", "Zip")
df.drop("Units", "Revenue", "Country")

### Trabajando con columnas

Para acceder a las columnas, debemos crear objetos [Column](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/column.html). Para ello, podemos seleccionarlos a partir de un DataFrame como una propiedad o mediante la función `col`:

In [0]:
# nomCliente = df.nombre
# nomCliente = df["ProductID"]
# nomCliente = col("ProductID")

Así pues, podemos recuperar ciertas columnas de un DataFrame con cualquier de las siguientes expresiones:

In [0]:
from pyspark.sql.functions import col

df.select("ProductID", "Revenue").show()
df.select(df.ProductID, df.Revenue).show()
df.select(df["ProductID"], df["Revenue"]).show()
df.select(col("ProductID"), col("Revenue")).show()

### col vs expr

En ocasiones se confunde el uso de la función [col ](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.col.html)con [expr](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.expr.html). Aunque podemos referenciar a una columna haciendo uso de `expr`, su uso provoca que se parseé la cadena recibida para interpretarla.

Para el siguiente ejemplo, supongamos que tenemos un DataFrame con datos de clientes. Utilizaremos también la función [concat_ws](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.concat_ws.html) para concatenar textos utilizado un separador.

In [0]:
df.printSchema()


In [0]:
from pyspark.sql.functions import col, concat_ws, expr

# Seleccionar columnas existentes y realizar operaciones
df.select(
    concat_ws(" ", col("Country"), col("Zip")).alias("CountryZip"),  # Concatenar Country y Zip
    col("Revenue"),                                                 # Mantener la columna Revenue
    expr("Revenue * 1.1").alias("AdjustedRevenue")                  # Calcular nueva columna Revenue ajustada
).show()


### Añadiendo columnas

Una vez tenemos un DataFrame, podemos añadir columnas mediante el método [withColumn](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html):

In [0]:
dfNuevo = df.withColumn("total", df.Units * df.Revenue)
dfNuevo.show()

> **withColumn**
> Anteriormente hemos utilizado el método [withColumn](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html) para cambiarle el tipo a un campo ya existente. Así pues, si referenciamos a una columna que ya existe, en vez de crearla, la sustituirá.

Otra forma de añadir una columna con una expresión es mediante la transformación [selectExpr](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.selectExpr.html). Por ejemplo, podemos conseguir el mismo resultado que en el ejemplo anterior de la siguiente manera:

In [0]:
df.selectExpr("*", "Units * Revenue as total").show()

Aunque más adelante veremos como realizar transformaciones con agregaciones, mediante `selectExpr` también podemos realizar analítica de datos aprovechando la potencia de SQL:

In [0]:
df.selectExpr("count(distinct(ProductID)) as productos","count(distinct(Country)) as paises").show()

**Cambiando el nombre**  
Si por algún extraño motivo necesitamos cambiarle el nombre a una columna (por ejemplo, vamos a unir dos DataFrames que tienen columnas con el mismo nombre pero en posiciones diferentes, o que al inferir el esquema tenga un nombre críptico o demasiado largo y queremos que sea más legible) podemos utilizar la transformación [withColumnRenamed](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumnRenamed.html):

In [0]:
df.withColumnRenamed("Zip", "PostalCode").show(5)

### Filtrando

Si queremos eliminar filas, usaremos el método [filter](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html):

In [0]:
df.filter(df.Country=="Germany").show()

Por similitud con SQL, podemos utilizar también `where` como un alias de `filter`:

In [0]:
df.where(df.Units>20).show()

Podemos utilizar los operadores lógicos (`&` para conjunción y `|` para la disyunción) para crear condiciones compuestas (recordad rodear cada condición entre paréntesis):

In [0]:
df.filter((df.Country=="Germany") & (df.Units>20)).show()

In [0]:
df.filter((df.ProductID==2314) | (df.ProductID==1322)).show()

Un caso particular de filtrado es la eliminación de los registros repetidos, lo cual lo podemos hacer de dos maneras:  

- Haciendo uso del método [distinct](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.distinct.html) tras haber realizado alguna transformación.  
- Utilizando [dropDuplicates](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.dropDuplicates.html) sobre un DataFrame:

In [0]:
df.select("Country").distinct().show()

In [0]:
df.dropDuplicates(["Country"]).select("Country").show()

### **Ordenando**  

Una vez recuperados los datos deseados, podemos ordenarlos mediante [sort](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.sort.html) u [orderBy](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.orderBy.html) (son operaciones totalmente equivalentes):

In [0]:
df.select("ProductID","Revenue").sort("Revenue").show(5)

In [0]:
df.sort("Revenue").show(5)

In [0]:
df.sort("Revenue", ascending=True).show(5)

In [0]:
df.sort(df.Revenue.asc()).show(5)

In [0]:
# Ordenación descendente
df.sort(df.Revenue.desc()).show(5)

In [0]:
df.sort("Revenue", ascending=False).show(5)

In [0]:
from pyspark.sql.functions import desc
df.sort(desc("Revenue")).show(5)

In [0]:
# Ordenación diferente en cada columna
df.sort(df.Revenue.desc(), df.Units.asc()).show(5)

In [0]:
df.sort(["Revenue","Units"], ascending=[0,1]).show(5)

Normalmente, tras realizar una ordenación, es habitual quedarse con un subconjunto de los datos. Para ello, podemos utilizar la transformación [limit](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.limit.html).  
Por ejemplo, la siguiente transformación es similar al ejemplo anterior, sólo que ahora al driver únicamente le llegan 5 registros, en vez de traerlos todos y sólo mostrar 5:

In [0]:
df.sort(df.Revenue.desc(), df.Units.asc()).limit(5).show()

### **Añadiendo filas**  

a única manera de añadir filas a un DataFrame es creando uno nuevo que sea el resultado de unir dos DataFrames que compartan el mismo esquema (mismo nombres de columnas y en el mismo orden). Para ello, utilizaremos la transformación [union](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.union.html) que realiza la unión por el orden de las columnas:

In [0]:
nuevasVentas = [
    (6666, "2022-03-24", "03206", 33, 3333.33, "Spain"),
    (6666, "2022-03-25", "03206", 22, 2222.22, "Spain"),
]

# Creamos un nuevo DataFrame con las nuevas ventas
nvDF = spark.createDataFrame(nuevasVentas)


In [0]:
# Unimos los dos DataFrames
dfUpdated = df.union(nvDF)

> **Trabajando con conjuntos**  
> Considerando dos DataFrames como dos conjuntos, podemos emplear las operaciones [union](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.union.html), [intersect](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.intersect.html), [intersectAll](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.intersectAll.html) (mantiene los duplicados), [exceptAll](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.exceptAll.html) (mantiene los duplicados) y [subtract](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.subtract.html) .

### Cogiendo muestras  
Si necesitamos recoger un subconjunto de los datos, ya sea para preparar los datos para algún modelo de machine learning como para una muestra aleatoria de los mismos, podemos utilizar las siguientes transformaciones:  
- [sample](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.sample.html) permite obtener una muestra a partir de un porcentaje (no tiene porqué obtener una cantidad exacta). También admite un semilla e indicar si queremos que pueda repetir los datos.

In [0]:
df.count()                  # 120239

In [0]:
muestra = df.sample(0.10)

In [0]:
muestra.count()             # 11876

In [0]:
muestraConRepetidos = df.sample(True, 0.10)

In [0]:
muestraConRepetidos.count() # 11923

- [randomSplit](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.randomSplit.html) recupera diferentes DataFrames cuyos tamaños en porcentaje se indican como parámetros (si no suman uno, los parámetros se normalizan):

In [0]:
dfs = df.randomSplit([0.8, 0.2])
dfEntrenamiento = dfs[0]

In [0]:
dfPrueba = dfs[1]

In [0]:
dfEntrenamiento.count()     # 96194

In [0]:
dfPrueba.count()            # 24045

## Trabajando con datos sucios

Hay tres formas de gestionar la suciedad de los datos o la omisión completa de los mismos:  
1. Eliminar las filas que tienen valores vacíos en una o más columnas.  
2. Rellenar los valores nulos con valores que definimos nosotros.  
3. Sustituir los datos erróneos por algún valor que sepamos cómo gestionarlo.

Vamos a ver cada uno de estos casos a partir del siguiente dataset:
  

In [0]:
malasVentas = [
    (6666, "2022-03-22", "03206", 33, 3333.33, "Spain"),
    (6666, "2022-03-22", None, 33, 3333.33, "Spain"),
    (6666, "2022-03-23", "03206", None, 2222.22, "Spain"),
    (6666, "2022-03-24", "03206", None, None, "Espain"),
    (None, None, None, None, None, None)
]
malDF = spark.createDataFrame(malasVentas, ["ProductID", "Date", "Zip", "Units", "Revenue" , "Country"])
malDF.show()

+---------+----------+-----+-----+-------+-------+
|ProductID|      Date|  Zip|Units|Revenue|Country|
+---------+----------+-----+-----+-------+-------+
|     6666|2022-03-22|03206|   33|3333.33|  Spain|
|     6666|2022-03-22| null|   33|3333.33|  Spain|
|     6666|2022-03-23|03206| null|2222.22|  Spain|
|     6666|2022-03-24|03206| null|   null| Espain|
|     null|      null| null| null|   null|   null|
+---------+----------+-----+-----+-------+-------+



Si queremos saber si una columna contiene nulos, podemos hacer un filtrado utilizando el método [isNull](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.isNull.html) sobre los campos deseados (también podemos utilizar [isNotNull](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.isNotNull.html) si queremos el caso contrario):

In [0]:
malDF.filter(malDF.Zip.isNull()).show()

Para trabajar con las filas que contengan algún dato nulo, podemos acceder a la propiedad `na`, la cual devuelve un [DataFrameNaFunctions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameNaFunctions.html) sobre la que podemos indicarle:  
- que la elimine mediante el método [drop](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameNaFunctions.drop.html) / [dropna](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.dropna.html). Puede recibir `"any"` (borrará las filas que contengan algún nulo) o `"all"` (borrará las filas que todas sus columnas contengan nulos) y una lista con las columnas a considerar:

In [0]:
# Elimina todos los nulos
malDF.na.drop().show()

In [0]:
# Elimina las filas que todas sus columnas son nulas
malDF.na.drop("all").show()

In [0]:
# Elimina las filas que tienen el Zip nulo
malDF.na.drop(subset=["Zip"]).show()

También podemos indicar la cantidad de valores no nulos que ha de contener cada fila para eliminarla mediante el parámetro `thresh`:

In [0]:
# Elimina las filas que tengan menos de 3 valores rellenados
malDF = malDF.dropna(thresh = 3)
malDF.show()

- que la rellene mediante el método [fill](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameNaFunctions.fill.html) / [fillna](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.fillna.html), indicando el valor y si queremos, sobre qué columnas aplicar la modificación:

In [0]:
# Rellenamos los zips vacíos por 99999
malDF.na.fill("99999", subset=["Zip"]).show()

- que la sustituya mediante el método [replace](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameNaFunctions.replace.html)

In [0]:
# Cambiamos Espain por Spain
malDF.na.replace("Espain", "Spain").show()

> **na.replace vs replace**  

La función `replace` se puede emplear dentro de la propiedad `na` o a partir del propio Dataframe.

Otro caso muy común es realizar una operación sobre una columna para transformar su valor, por ejemplo, pasar todo el texto a minúsculas o dividir una columna entre 100 para cambiar la escala.  
En nuestro caso, vamos a modificar las columnas Zip y Country para realizar un `trim` y borrar los espacios en blanco:

In [0]:
from pyspark.sql.functions import col, trim
df = df.withColumn("Country", trim(col("Country"))).withColumn("Zip", trim(col("Zip")))
df.show()

## Usando SQL

En la era del big data SQL es la lengua franca, permitiendo a perfiles con pocos conocimientos de programación trabajar de forma eficiente con los datos (siempre poniendo el foco en la analítica de datos, no en el procesamiento transaccional). Spark soporta el ANSI SQL 2003, ampliamente establecido en el mundo de las bases de datos. Para correr SQL en Spark podemos hacerlo a través de:  
- El cliente SQL, es cual se ofrece como un comando en `./bin/spark-sql`  
- Mediante un servidor ODBC/JDBC  
- De forma programativa mediante aplicaciones Spark.  

Las dos primeras opciones se integran con Apache Hive para utilizar su metastore. Ahora nos vamos a centrar en la última.

### Vistas temporales  
Ya hemos visto que los DataFrames tienen una estructura similar a una tabla de una base de datos relacional. Para poder realizar consultas, necesitaremos crear vistas temporales mediante el método [createTempView](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.createTempView.html) o [createOrReplaceTempView](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.createOrReplaceTempView.html) para posteriormente realizar una consulta sobre la vista creada a través de [spark.sql](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.sql.html):

In [0]:
# 1. definimos la vista
df.createOrReplaceTempView("ventas")

In [0]:
# 2. realizamos la consulta
ventasCanada = spark.sql("select * from ventas where trim(Country)='Canada'")
ventasCanada.show(3)

### Vistas globales  
Las vistas temporales tienen un alcance de SparkSession, de manera que desaparecen una vez finalice la sesión que ha creado la vista. Si necesitamos tener una vista que se comparta entre todas las sesiones y que permanezca viva hasta que la aplicación Spark finalice, podemos crear una vista temporal global mediante [createOrReplaceGlobalTempView](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.createOrReplaceGlobalTempView.html)

Estas vistas se almacenan en la base de datos `global_temp` y en las consultas es necesario poner el prefijo `global_temp` para acceder a sus vistas.

In [0]:
# 1. definimos la vista global
df.createOrReplaceGlobalTempView("ventasg")

In [0]:
# 2. realizamos la consulta
ventasCanadaG = spark.sql("select * from global_temp.ventasg where trim(Country)='Canada'")
ventasCanadaG.show(3)

In [0]:
# Creamos otra sesión y vemos como funciona
spark.newSession().sql("select count(*) from global_temp.ventasg where trim(Country)='Canada'").show()

### Eliminando vistas  
Para borrar una vista que hayamos creado, necesitamos acceder al Spark Catalog que veremos en una [sesión posterior(catalog#spark-sql-catalog), y utilizar el método [dropTempView](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Catalog.dropTempView.html) o [dropGlobalTempView](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Catalog.dropGlobalTempView.html) dependiendo del tipo de vista:

In [0]:
spark.catalog.dropTempView("ventas")
spark.catalog.dropGlobalTempView("ventasg")

Out[23]: False

## Trabajando con Databricks y SQL  
Vamos a cargar en `DataBase Tables` el csv `pdi_sales_small.csv`. Para ellos vamos a crear primero la tabla. Click en Catalog

![](files/dataframes-spark/images/1.png)


![](files/dataframes-spark/images/2.png)

![](files/dataframes-spark/images/3.png)

![](files/dataframes-spark/images/4.png)

![](files/dataframes-spark/images/5.png)

Para que funcione correctamente con nuestro datos, vamos a modificar el código:

```<Python>
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"
```

Y tras cargar el dataset, antes de crear la vista, vamos a limpiar los países:

```<python>
from pyspark.sql.functions import trim
df = df.withColumn("Country", trim(df.Country))
```

### Datos visuales

Si volvemos a ejecutar el cuaderno, ahora sí que cargará correctamente los datos. Si nos vamos a la celda que realiza una consulta sobre todos los datos, podemos ver en la parte superior derecha como el lenguaje empleado en la celda es SQL, por ello la primera línea comienza con %sql, y a continuación ya podemos introducir directamente código SQL, teniendo la opción de visualizar los datos tanto en modo texto como mediante gráficos:

In [0]:
%sql
select * from `pdi_sales_small` 

![](files/dataframes-spark/images/6.png)

![](files/dataframes-spark/images/7.png)

![](files/dataframes-spark/images/8.png)

![](files/dataframes-spark/images/9.png)

![](files/dataframes-spark/images/10.png)

![](files/dataframes-spark/images/11.png)

### Cuadro de mandos

Además, con las tablas y/o gráficos que generamos dentro de Databricks, podemos generar un sencillo cuadro de mandos.

Vamos a crear un par de consultas, una para obtener las ventas medias por país:

```<sql>
%sql
select Country, avg(Revenue) as ventas
from pdi_sales_small_csv
group by Country
order by ventas desc
```

Y otra para las unidas pedidas por cada país:

```<sql>
%sql
select Country, sum(Units) as pedidos
from pdi_sales_small_csv
group by Country
order by pedidos desc
```

Si pulsamos sobre el icono del `+` , podemos añadir el resultado de la celda a un dashboard:

![](files/dataframes-spark/images/12.png)

![](files/dataframes-spark/images/13.png)

![](files/dataframes-spark/images/14.png)

![](files/dataframes-spark/images/15.png)

![](files/dataframes-spark/images/16.png)

![](files/dataframes-spark/images/17.png)

![](files/dataframes-spark/images/18.png)

![](files/dataframes-spark/images/19.png)

![](files/dataframes-spark/images/20.png)

![](files/dataframes-spark/images/21.png)

## Actividades

En las siguientes actividades vamos a familiarizarnos con el uso del API de DataFrames de Spark.  
1. A partir del archivo [nombres.json](https://tajamar365.sharepoint.com/:u:/s/3405-MasterIA2024-2025/ESw3v7IfZAFNqa7Kb05kuu0Bz7hbNQOvX2ZsbtGskFjuJA?e=UUJrbF), crea un DataFrame y realiza las siguientes operaciones:  
a. Crea una nueva columna (columna `Mayor30`) que indique si la persona es mayor de 30 años.  


In [0]:
from pyspark.sql.functions import col, lit, current_date, year, monotonically_increasing_id

# Carga del archivo JSON
df = spark.read.json("/FileStore/dataframes-spark/dataset/nombres.json")
df.show()


+--------+----+------+
|  Ciudad|Edad|Nombre|
+--------+----+------+
|   Elche|  45| Aitor|
|Alicante|  14|Marina|
|   Elche|  19| Laura|
|    Aspe|  45| Sonia|
|   Elche|null| Pedro|
+--------+----+------+



b. Crea una nueva columna (columna `FaltanJubilacion`) que calcule cuantos años le faltan para jubilarse (supongamos que se jubila a los 67 años).  


In [0]:
df = df.withColumn("FaltanJubilacion", lit(67) - col("Edad"))
df.show()

+--------+----+------+----------------+
|  Ciudad|Edad|Nombre|FaltanJubilacion|
+--------+----+------+----------------+
|   Elche|  45| Aitor|              22|
|Alicante|  14|Marina|              53|
|   Elche|  19| Laura|              48|
|    Aspe|  45| Sonia|              22|
|   Elche|null| Pedro|            null|
+--------+----+------+----------------+



c. Crea una nueva columna (columna `Apellidos`) que contenga XYZ (puedes utilizar la función [lit](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.lit.html))  


In [0]:
df = df.withColumn("Apellidos", lit("XYZ"))
df.show()


+--------+----+------+----------------+---------+
|  Ciudad|Edad|Nombre|FaltanJubilacion|Apellidos|
+--------+----+------+----------------+---------+
|   Elche|  45| Aitor|              22|      XYZ|
|Alicante|  14|Marina|              53|      XYZ|
|   Elche|  19| Laura|              48|      XYZ|
|    Aspe|  45| Sonia|              22|      XYZ|
|   Elche|null| Pedro|            null|      XYZ|
+--------+----+------+----------------+---------+



In [0]:
df = df.drop("Mayor30", "Apellidos")
df.show()


+--------+----+------+----------------+
|  Ciudad|Edad|Nombre|FaltanJubilacion|
+--------+----+------+----------------+
|   Elche|  45| Aitor|              22|
|Alicante|  14|Marina|              53|
|   Elche|  19| Laura|              48|
|    Aspe|  45| Sonia|              22|
|   Elche|null| Pedro|            null|
+--------+----+------+----------------+



d. Elimina las columna `Mayor30` y `Apellidos`.  


In [0]:
df = df.drop("Mayor30", "Apellidos")
df.show()

+--------+----+------+----------------+
|  Ciudad|Edad|Nombre|FaltanJubilacion|
+--------+----+------+----------------+
|   Elche|  45| Aitor|              22|
|Alicante|  14|Marina|              53|
|   Elche|  19| Laura|              48|
|    Aspe|  45| Sonia|              22|
|   Elche|null| Pedro|            null|
+--------+----+------+----------------+



e. Crea una nueva columna (columna `AnyoNac`) con el año de nacimiento de cada persona (puedes utilizar la función [current_date](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.current_date.html)).  


In [0]:
df = df.withColumn("AnyoNac", year(current_date()) - col("Edad"))
df.show()

+--------+----+------+----------------+-------+
|  Ciudad|Edad|Nombre|FaltanJubilacion|AnyoNac|
+--------+----+------+----------------+-------+
|   Elche|  45| Aitor|              22|   1979|
|Alicante|  14|Marina|              53|   2010|
|   Elche|  19| Laura|              48|   2005|
|    Aspe|  45| Sonia|              22|   1979|
|   Elche|null| Pedro|            null|   null|
+--------+----+------+----------------+-------+



f. Añade un id incremental para cada fila (campo `Id`) y haz que al hacer un `show` se vea en primer lugar (puedes utilizar la función [monotonically_increasing_id](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.monotonically_increasing_id.html)) seguidos del `Nombre`, `Edad`, `AnyoNac`, `FaltaJubilacion` y `Ciudad`

In [0]:
df = df.withColumn("Id", monotonically_increasing_id())
df = df.select("Id", "Nombre", "Edad", "AnyoNac", "FaltanJubilacion", "Ciudad")
df.show()

+---+------+----+-------+----------------+--------+
| Id|Nombre|Edad|AnyoNac|FaltanJubilacion|  Ciudad|
+---+------+----+-------+----------------+--------+
|  0| Aitor|  45|   1979|              22|   Elche|
|  1|Marina|  14|   2010|              53|Alicante|
|  2| Laura|  19|   2005|              48|   Elche|
|  3| Sonia|  45|   1979|              22|    Aspe|
|  4| Pedro|null|   null|            null|   Elche|
+---+------+----+-------+----------------+--------+



Al realizar los seis pasos, el resultado del DataFrame será similar a :

![](files/dataframes-spark/images/77.png)![](path)

2. A partir del archivo [VentasNulos.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EUP8X8fQMpNJv5DNBzu9cXUBNy5gSjYkjiFb5yyAfyvtfw?e=eruozp):  
a. Elimina las filas que tengan al menos 4 nulos.  
 


In [0]:
from pyspark.sql.functions import col, lit, when, round, avg, min

# Carga del archivo CSV
df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/dataframes-spark/dataset/VentasNulos.csv")
df.show()


+------+------+-----+-----------+-------------+
|Nombre|Ventas|Euros|     Ciudad|Identificador|
+------+------+-----+-----------+-------------+
|  Pepe|     4|  200|      Elche|          X21|
|Andreu|     8| null|       null|         null|
|  Juan|  null| null|       null|          C54|
| Pedro|     1|   30|   Valencia|          R23|
| María|  null|  300| Torrellano|         null|
|Marina|     3|  350|       Aspe|          V55|
|  null|    10|  500|Crevillente|          AMV|
|   Ana|    10| 2300|   Alicante|          B89|
|  null|  null| null|       null|         null|
| Jorge|     8| null|       null|          T19|
+------+------+-----+-----------+-------------+



b. Con las filas restantes, sustituye: 

Para los pasos ii) y iii)  puedes crear un DataFrame que obtenga el valor a asignar y luego pasarlo como parámetro al método para rellenar los nulos. 
  
  i. Los nombres nulos por Empleado  
  

In [0]:
df = df.withColumn("Nombre", when(col("Nombre").isNull(), "Empleado").otherwise(col("Nombre")))
df.show()


+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8| null|       null|         null|
|    Juan|  null| null|       null|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|  null|  300| Torrellano|         null|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|  null| null|       null|         null|
|   Jorge|     8| null|       null|          T19|
+--------+------+-----+-----------+-------------+



ii. Las ventas nulas por la media de las ventas de los compañeros (redondeado a entero).
      > **Agrupando:** Para obtener la media, aunque lo veremos en la próxima sesión, debes agrupar y luego obtener la media de la columna:  
      > `valor = df.groupBy().avg('Ventas')`   

In [0]:
# Calcula la media de las ventas
media_ventas = df.groupBy().agg(round(avg("Ventas"))).collect()[0][0]

# Sustituye los nulos en la columna Ventas por la media calculada
df = df.withColumn("Ventas", when(col("Ventas").isNull(), lit(media_ventas)).otherwise(col("Ventas")))
df.show()


+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|   4.0|  200|      Elche|          X21|
|  Andreu|   8.0| null|       null|         null|
|    Juan|   6.0| null|       null|          C54|
|   Pedro|   1.0|   30|   Valencia|          R23|
|   María|   6.0|  300| Torrellano|         null|
|  Marina|   3.0|  350|       Aspe|          V55|
|Empleado|  10.0|  500|Crevillente|          AMV|
|     Ana|  10.0| 2300|   Alicante|          B89|
|Empleado|   6.0| null|       null|         null|
|   Jorge|   8.0| null|       null|          T19|
+--------+------+-----+-----------+-------------+



iii. Los euros nulos por el valor del compañero que menos € ha ganado. (tras agrupar, puedes usar la función `min`)  
    

In [0]:
# Calcula el valor mínimo de Euros
min_euros = df.groupBy().agg(min("Euros")).collect()[0][0]

# Sustituye los nulos en la columna Euros por el mínimo calculado
df = df.withColumn("Euros", when(col("Euros").isNull(), lit(min_euros)).otherwise(col("Euros")))
df.show()


+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|   4.0|  200|      Elche|          X21|
|  Andreu|   8.0|   30|       null|         null|
|    Juan|   6.0|   30|       null|          C54|
|   Pedro|   1.0|   30|   Valencia|          R23|
|   María|   6.0|  300| Torrellano|         null|
|  Marina|   3.0|  350|       Aspe|          V55|
|Empleado|  10.0|  500|Crevillente|          AMV|
|     Ana|  10.0| 2300|   Alicante|          B89|
|Empleado|   6.0|   30|       null|         null|
|   Jorge|   8.0|   30|       null|          T19|
+--------+------+-----+-----------+-------------+



iv. La ciudad nula por `C.V`. y el identificador nulo por `XYZ`.  


In [0]:
df = df.withColumn("Ciudad", when(col("Ciudad").isNull(), "C.V.").otherwise(col("Ciudad")))
df = df.withColumn("Identificador", when(col("Identificador").isNull(), "XYZ").otherwise(col("Identificador")))
df.show()



+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|   4.0|  200|      Elche|          X21|
|  Andreu|   8.0|   30|       C.V.|          XYZ|
|    Juan|   6.0|   30|       C.V.|          C54|
|   Pedro|   1.0|   30|   Valencia|          R23|
|   María|   6.0|  300| Torrellano|          XYZ|
|  Marina|   3.0|  350|       Aspe|          V55|
|Empleado|  10.0|  500|Crevillente|          AMV|
|     Ana|  10.0| 2300|   Alicante|          B89|
|Empleado|   6.0|   30|       C.V.|          XYZ|
|   Jorge|   8.0|   30|       C.V.|          T19|
+--------+------+-----+-----------+-------------+



3.  A partir del archivo [movies.tsv](https://tajamar365.sharepoint.com/:u:/s/3405-MasterIA2024-2025/Ef-LbTw0tw1JgYAXrl-zdQQBw73LyG0oknmFzxbs01CL8w?e=AC8asA), crea una esquema de forma declarativa con los campos:  


In [0]:
df1 = spark.read.format("csv").option("delimiter", "\t").option("header", "true").load("dbfs:/FileStore/dataframes-spark/dataset/movies.tsv")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Definir el esquema
schema = StructType([
    StructField("interprete", StringType(), True),
    StructField("pelicula", StringType(), True),
    StructField("anyo", IntegerType(), True)
])


- Cargar los datos en un DataFrame
Carga el archivo movies.tsv usando el esquema definido. Asegúrate de que el archivo está en el formato correcto.

In [0]:

# Cargar datos desde el archivo TSV
df = spark.read \
    .option("header", False) \
    .option("sep", "\t") \
    .schema(schema) \
    .csv("/FileStore/dataframes-spark/dataset/movies.tsv")

df.show(truncate=False)


+-----------------+---------------------------+----+
|interprete       |pelicula                   |anyo|
+-----------------+---------------------------+----+
|McClure, Marc (I)|Freaky Friday              |2003|
|McClure, Marc (I)|Coach Carter               |2005|
|McClure, Marc (I)|Superman II                |1980|
|McClure, Marc (I)|Apollo 13                  |1995|
|McClure, Marc (I)|Superman                   |1978|
|McClure, Marc (I)|Back to the Future         |1985|
|McClure, Marc (I)|Back to the Future Part III|1990|
|Cooper, Chris (I)|Me, Myself & Irene         |2000|
|Cooper, Chris (I)|October Sky                |1999|
|Cooper, Chris (I)|Capote                     |2005|
|Cooper, Chris (I)|The Bourne Supremacy       |2004|
|Cooper, Chris (I)|The Patriot                |2000|
|Cooper, Chris (I)|The Town                   |2010|
|Cooper, Chris (I)|Seabiscuit                 |2003|
|Cooper, Chris (I)|A Time to Kill             |1996|
|Cooper, Chris (I)|Where the Wild Things Are  

Cada fila del fichero implica que el actor/actriz ha trabajado en dicha película en el año indicado. 

a. Muestra las películas en las que ha trabajado Murphy, Eddie (I)

In [0]:
peliculas_murphy = df.filter(df.interprete == "Murphy, Eddie (I)").select("pelicula")
peliculas_murphy.show(truncate=False)


+------------------------------+
|pelicula                      |
+------------------------------+
|Showtime                      |
|Norbit                        |
|Hot Tub Time Machine          |
|Nutty Professor II: The Klumps|
|Beverly Hills Cop II          |
|Trading Places                |
|Daddy Day Care                |
|Dr. Dolittle 2                |
|Shrek Forever After           |
|Beverly Hills Cop             |
|Shrek                         |
|The Haunted Mansion           |
|Coming to America             |
|Shrek 2                       |
|Doctor Dolittle               |
|The Nutty Professor           |
|Mulan                         |
|Tower Heist                   |
|Dreamgirls                    |
|Bowfinger                     |
+------------------------------+
only showing top 20 rows



b. Muestra los intérpretes que aparecen tanto en Superman como en Superman II.

In [0]:
# Filtrar los intérpretes en "Superman"
superman_interpretes = df.filter(df.pelicula == "Superman").select("interprete")

# Filtrar los intérpretes en "Superman II"
superman_ii_interpretes = df.filter(df.pelicula == "Superman II").select("interprete")

# Encontrar los intérpretes comunes entre ambas películas
interpretes_comunes = superman_interpretes.intersect(superman_ii_interpretes)
interpretes_comunes.show(truncate=False)


+------------------+
|interprete        |
+------------------+
|O'Halloran, Jack  |
|Tucker, Burnell   |
|Hollis, John (I)  |
|Beatty, Ned       |
|Stamp, Terence    |
|Ratzenberger, John|
|Hackman, Gene     |
|Fielder, Harry    |
|Perrine, Valerie  |
|McClure, Marc (I) |
|Donner, Richard   |
+------------------+



#### Conclusión
Esta práctica demuestra cómo usar el API de DataFrames en PySpark para trabajar con datos estructurados, aplicando esquemas personalizados, filtrado, transformaciones, y operaciones de conjunto. Permite entender cómo manejar grandes volúmenes de datos, identificar patrones (como películas comunes) y realizar análisis específicos de manera eficiente y escalable. Es ideal para iniciar en análisis de datos distribuidos.